In [996]:
using PolyChaos
using NLsolve
using LinearAlgebra

In [997]:
P = 5
op = GaussOrthoPoly(P, Nrec=100, addQuadrature=true)
t3 = Tensor(3, op)
M = [t3.get([i, j, k]) for i=0:P, j=0:P, k=0:P]

6×6×6 Array{Float64, 3}:
[:, :, 1] =
 1.0  0.0  0.0  0.0   0.0    0.0
 0.0  1.0  0.0  0.0   0.0    0.0
 0.0  0.0  2.0  0.0   0.0    0.0
 0.0  0.0  0.0  6.0   0.0    0.0
 0.0  0.0  0.0  0.0  24.0    0.0
 0.0  0.0  0.0  0.0   0.0  120.0

[:, :, 2] =
 0.0  1.0  0.0   0.0    0.0    0.0
 1.0  0.0  2.0   0.0    0.0    0.0
 0.0  2.0  0.0   6.0    0.0    0.0
 0.0  0.0  6.0   0.0   24.0    0.0
 0.0  0.0  0.0  24.0    0.0  120.0
 0.0  0.0  0.0   0.0  120.0    0.0

[:, :, 3] =
 0.0  0.0   2.0    0.0    0.0     0.0
 0.0  2.0   0.0    6.0    0.0     0.0
 2.0  0.0   8.0    0.0   24.0     0.0
 0.0  6.0   0.0   36.0    0.0   120.0
 0.0  0.0  24.0    0.0  192.0     0.0
 0.0  0.0   0.0  120.0    0.0  1200.0

[:, :, 4] =
 0.0   0.0    0.0    6.0     0.0     0.0
 0.0   0.0    6.0    0.0    24.0     0.0
 0.0   6.0    0.0   36.0     0.0   120.0
 6.0   0.0   36.0    0.0   216.0     0.0
 0.0  24.0    0.0  216.0     0.0  1440.0
 0.0   0.0  120.0    0.0  1440.0     0.0

[:, :, 5] =
  0.0    0.0    0.0     0.0  

In [998]:
b = randn(P+1)
c = randn(P+1)

6-element Vector{Float64}:
 -0.7923558717887956
  0.43916116106907216
  0.6259533539778089
 -0.7149548108102166
  1.1539068842427276
  0.4187212436423394

In [999]:
d = zeros(P+1)
for k in 1:P+1
    for i in 1:P+1
        for j in 1:P+1
            d[k] += (b[i]*b[j] + c[i]*c[j])*M[i, j, k]
        end
    end
end
d

6-element Vector{Float64}:
   147.15916411781348
   121.01067223053505
  1093.373844007278
  1870.5351628291885
 11871.082149180062
 20164.296069853335

In [1000]:
function F!(F, a)
    for k in 1:P+1
        F[k] = 0.0
        for i in 1:P+1
            for j in 1:P+1
                F[k] += a[i]*a[j]*M[i, j, k]
            end
        end
        F[k] -= d[k]
    end
end

F! (generic function with 1 method)

In [1001]:
function j!(J, a)
    for k in 1:P+1
        for i in 1:P+1
            J[k, i] = 0.0
            for j in 1:P+1
                J[k, i] += a[j]*(M[i, j, k] + M[j, i, k])
            end
        end
    end
end

j! (generic function with 1 method)

In [1002]:
a0 = sqrt.(b.^2 + c.^2)

6-element Vector{Float64}:
 1.3230267406990166
 0.5528380618920514
 0.7190751879271404
 1.2603296861423263
 1.2140109681245355
 0.9090593469014987

In [1003]:
sol = nlsolve(F!, j!, a0)

Results of Nonlinear Solver Algorithm
 * Algorithm: Trust-region with dogleg and autoscaling
 * Starting Point: [1.3230267406990166, 0.5528380618920514, 0.7190751879271404, 1.2603296861423263, 1.2140109681245355, 0.9090593469014987]
 * Zero: [6.301891097714429, 0.31647928451739993, 2.248435280800496, -0.10680784527413897, 0.5804899838780163, 0.8615807020668379]
 * Inf-norm of residuals: 0.000000
 * Iterations: 9
 * Convergence: true
   * |x - x'| < 0.0e+00: false
   * |f(x)| < 1.0e-08: true
 * Function Calls (f): 10
 * Jacobian Calls (df/dx): 10

In [1004]:
sol.zero

6-element Vector{Float64}:
  6.301891097714429
  0.31647928451739993
  2.248435280800496
 -0.10680784527413897
  0.5804899838780163
  0.8615807020668379